In [1]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_d90d9e995ba049108de1176f9a6e23b8.csv"
df=pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1


In [3]:
df.shape

(1473, 10)

In [4]:
age=pd.DataFrame(df.iloc[:,0]).rename(columns={0:"age"})
age.head()

,age
0,24
1,45
2,43
3,42
4,36


In [5]:
from sklearn.preprocessing import KBinsDiscretizer
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='uniform')
age=pd.DataFrame(est.fit_transform(age),columns=age.columns)
age.head()

,age
0,0.0
1,3.0
2,3.0
3,3.0
4,2.0


In [6]:
#child=pd.DataFrame(df.iloc[:,3]).rename(columns={3:"child"})
#child.head()

In [7]:
#child=pd.DataFrame(est.fit_transform(child),columns=child.columns)
#child.head()

In [8]:
y=pd.DataFrame(df.iloc[:,9]).rename(columns={9:"Class"})
y.groupby("Class")["Class"].count()

Class
1    629
2    333
3    511
Name: Class, dtype: int64

In [9]:
sub_df=df.drop(columns=[0,9])
sub_df.head()

,1,2,3,4,5,6,7,8
0,2,3,3,1,1,2,3,0
1,1,3,10,1,1,3,4,0
2,2,3,7,1,1,3,4,0
3,3,2,9,1,1,3,3,0
4,3,3,8,1,1,3,2,0


In [10]:
X=pd.concat([age,sub_df],axis=1)
X.head()

,age,1,2,3,4,5,6,7,8
0,0.0,2,3,3,1,1,2,3,0
1,3.0,1,3,10,1,1,3,4,0
2,3.0,2,3,7,1,1,3,4,0
3,3.0,3,2,9,1,1,3,3,0
4,2.0,3,3,8,1,1,3,2,0


In [11]:

import numpy as np
from numpy import set_printoptions
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif


test = SelectKBest(score_func=f_classif, k="all")
fit = test.fit(X, np.asarray(y).reshape(y.shape[0],))
features = fit.transform(X)
set_printoptions(precision=3)

scores=fit.scores_
scores=pd.DataFrame(scores)
scores=scores.rename(columns={0:"scores"})
scores=scores.sort_values(by=["scores"],ascending=False)
scores

,scores
1,71.853562
7,31.509675
0,30.864459
6,26.827130
2,25.206981
8,16.099058
3,13.452832
4,10.949670
5,2.597444


In [12]:
X_scores=scores[scores["scores"]>=0]
len(X_scores)

9

In [13]:
X_prime=X.iloc[:,X_scores.index]
X_prime.head()

,1,7,age,6,2,8,3,4,5
0,2,3,0.0,2,3,0,3,1,1
1,1,4,3.0,3,3,0,10,1,1
2,2,4,3.0,3,3,0,7,1,1
3,3,3,3.0,3,2,0,9,1,1
4,3,2,2.0,3,3,0,8,1,1


In [14]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_prime,y,test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)

(1178, 9)
(295, 9)


In [15]:
from imblearn.ensemble import BalancedRandomForestClassifier
BForest=BalancedRandomForestClassifier(min_samples_split=3,min_samples_leaf=3,n_jobs=-1,
                                       random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BForest

BalancedRandomForestClassifier(min_samples_leaf=3, min_samples_split=3,
                               n_jobs=-1, random_state=0)

In [16]:
from sklearn.metrics import accuracy_score
train_pred=BForest.predict(X_train)
test_pred=BForest.predict(X_test)
print("Train data accuracy: ",accuracy_score(y_train,train_pred))
print("Test data accuracy: ",accuracy_score(y_test,test_pred))

Train data accuracy:  0.669779286926995
Test data accuracy:  0.6067796610169491


In [49]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
BBagging=BalancedBaggingClassifier(base_estimator=svm.SVC(C=2),sampling_strategy="all",n_jobs=-1,
                                   random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
BBagging

BalancedBaggingClassifier(base_estimator=SVC(C=2), n_jobs=-1, random_state=0,
                          sampling_strategy='all')

In [50]:
train_pred=BBagging.predict(X_train)
test_pred=BBagging.predict(X_test)
print("Train data accuracy: ",accuracy_score(y_train,train_pred))
print("Test data accuracy: ",accuracy_score(y_test,test_pred))

Train data accuracy:  0.5721561969439728
Test data accuracy:  0.6203389830508474


In [51]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix: \n", confusion_matrix(y_test,test_pred))

Confusion matrix: 
 [[80 24 26]
 [11 48 12]
 [13 26 55]]


In [20]:
Ada=AdaBoostClassifier(random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Ada

AdaBoostClassifier(random_state=0)

In [21]:
train_pred=Ada.predict(X_train)
test_pred=Ada.predict(X_test)
print("Train data accuracy: ",accuracy_score(y_train,train_pred))
print("Test data accuracy: ",accuracy_score(y_test,test_pred))

Train data accuracy:  0.566213921901528
Test data accuracy:  0.576271186440678


In [22]:
Gradient=GradientBoostingClassifier(learning_rate=0.025,n_estimators=90,criterion="mse",max_features="sqrt",
                                    random_state=0).fit(X_train,np.asarray(y_train).reshape(y_train.shape[0],))
Gradient

GradientBoostingClassifier(criterion='mse', learning_rate=0.025,
                           max_features='sqrt', n_estimators=90,
                           random_state=0)

In [23]:
train_pred=Gradient.predict(X_train)
test_pred=Gradient.predict(X_test)
print("Train data accuracy: ",accuracy_score(y_train,train_pred))
print("Test data accuracy: ",accuracy_score(y_test,test_pred))

Train data accuracy:  0.5967741935483871
Test data accuracy:  0.6372881355932203


In [25]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for test data: \n",confusion_matrix(y_test,test_pred))

Confusion matrix for test data: 
 [[89  6 35]
 [12 25 34]
 [15  5 74]]


In [26]:
from sklearn.metrics import classification_report
print("Classification report for test data: \n",classification_report(y_test,test_pred))

Classification report for test data: 
               precision    recall  f1-score   support

           1       0.77      0.68      0.72       130
           2       0.69      0.35      0.47        71
           3       0.52      0.79      0.62        94

    accuracy                           0.64       295
   macro avg       0.66      0.61      0.61       295
weighted avg       0.67      0.64      0.63       295



#### Conclusion: 
1. Do not standardize the dataset. Instead, discretize the only continuous variable, which is "age", and leave everything else as it is. Experiments, although not shown above, showed that this is the data manipulations strategy that resulted in best performances for al models.
2. **GradientBoostingClassifier" is the best model.**